In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib
import dask.dataframe as dd
import pandas as pd

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2022-01-18_lDE20_Final_5/Growth_Division/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_1 = infile["mCherry"][:]
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/Growth_Division/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_2 = infile["mCherry"][:]

In [ ]:
seg_1_handle = tr.segment.fluo_segmentation_cluster(
    "/home/de64/scratch/de64/sync_folder/2022-01-18_lDE20_Final_5/Growth_Division/",
    paramfile=True,
)
seg_2_handle = tr.segment.fluo_segmentation_cluster(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/Growth_Division/",
    paramfile=True,
)

In [ ]:
plt.imshow(stack_2[0])
plt.show()

In [ ]:
plt.imshow(stack_1[0] > 1500)
plt.show()

In [ ]:
stack_2.shape

In [ ]:
seg_1 = stack_1 > 1500

In [ ]:
seg_2 = stack_2 > 1500

In [ ]:
import skimage as sk

In [ ]:
label_1 = np.array([sk.measure.label(seg_1[i]) for i in range(seg_1.shape[0])])

In [ ]:
label_2 = np.array([sk.measure.label(seg_2[i]) for i in range(seg_2.shape[0])])

In [ ]:
label_1.shape

In [ ]:
props_1 = [
    sk.measure.regionprops(label_1[i], intensity_image=stack_1[i])
    for i in range(label_1.shape[0])
]
props_1 = [obj.intensity_mean for item in props_1 for obj in item]

props_2 = [
    sk.measure.regionprops(label_2[i], intensity_image=stack_2[i])
    for i in range(label_2.shape[0])
]
props_2 = [obj.intensity_mean for item in props_2 for obj in item]

In [ ]:
plt.hist(props_1, bins=100)
plt.show()

In [ ]:
plt.hist(props_2, bins=100)
plt.show()

In [ ]:
plt.hist(props_1, bins=100, alpha=0.4, density=True, log=True)
plt.hist(props_2, bins=100, alpha=0.4, density=True, log=True)
plt.show()

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="01:00:00",
    local=False,
    n_workers=50,
    n_workers_min=20,
    memory="16GB",
    working_directory="/home/de64/scratch/de64/dask",
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
import dask.dataframe as dd

lineage_df_1 = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2022-01-18_lDE20_Final_5/Growth_Division/lineage/"
)
lineage_df_2 = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/Growth_Division/lineage/"
)

In [ ]:
mchy_1 = lineage_df_1["mCherry mean_intensity"].sample(frac=0.01).compute()
mchy_2 = lineage_df_2["mCherry mean_intensity"].sample(frac=0.01).compute()

In [ ]:
plt.hist(mchy_1.values, bins=50)
plt.show()

In [ ]:
plt.hist(mchy_2.values, bins=50)
plt.show()

In [ ]:
plt.hist(
    mchy_1.values, bins=100, alpha=0.4, density=True, log=True, range=(2000, 12000)
)
plt.hist(
    mchy_2.values, bins=100, alpha=0.4, density=True, log=True, range=(2000, 12000)
)
plt.show()

- Confirmed that the mean seems to be a bit different between the experiments (at least in this FOV); this shouldn't be too suprising.
- Checking lineage dfs..
- Checking final lineage and barcode dfs
- Now suspicous that the barcodes in the first experiment (2022-01-18_lDE20_Final_5) were misread or otherwise scrambled. Re-running pipeline to check if calling one of the bits differently (and consistant with the other experiment) fixes it
    - Evidence was that shuffling the labels on the second experiment reproduced the distributions from the first
- If re-run fails, then need to investigate if there is a point in the pipeline where labels could get scrambled?
<!-- - Should now rerun the 2nd dataset with the altered value for kymograph param -->

In [ ]:
final_lineage_df1 = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2022-01-18_lDE20_Final_5/2022-02-15_lDE20_Final_Lineage_df/"
)
final_lineage_df2 = dd.read_parquet(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/2022-02-09_lDE20_Final_Lineage_df/"
)

In [ ]:
mchy1 = final_lineage_df1["Mean: mCherry Intensity"].sample(frac=0.01).compute()
mchy2 = final_lineage_df2["Mean: mCherry Intensity"].sample(frac=0.01).compute()

In [ ]:
plt.hist(mchy1, bins=100, alpha=0.4, density=True, log=True)
plt.hist(mchy2, bins=100, alpha=0.4, density=True, log=True)
plt.show()

In [ ]:
gene_cluster_df_full_w_control = pd.read_pickle(
    "/home/de64/scratch/de64/sync_folder/2022-01-18_lDE20_Final_5/2022-02-15_gene_cluster_df_no_filter.pkl"
)
gene_cluster_df_full = gene_cluster_df_full_w_control.dropna(
    subset=["Gene"]
)  # no control genes

gene_cluster_df_full_w_control_2 = pd.read_pickle(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/2022-02-10_gene_cluster_df_no_filter.pkl"
)
gene_cluster_df_full_2 = gene_cluster_df_full_w_control_2.dropna(
    subset=["Gene"]
)  # no control genes

In [ ]:
plt.hist(
    gene_cluster_df_full["Mean: mCherry Intensity"],
    bins=100,
    alpha=0.4,
    density=True,
    log=True,
)
plt.hist(
    gene_cluster_df_full_2["Mean: mCherry Intensity"],
    bins=100,
    alpha=0.4,
    density=True,
    log=True,
)
plt.show()

In [ ]:
import numpy as np

In [ ]:
mchy_traces_1 = np.stack(
    gene_cluster_df_full["Kernel Trace: Mean: mCherry Intensity"].tolist()
)
mchy_traces_2 = np.stack(
    gene_cluster_df_full_2["Kernel Trace: Mean: mCherry Intensity"].tolist()
)

In [ ]:
plt.hist(mchy_traces_1.flatten(), bins=100, alpha=0.4, density=True, log=True)
plt.hist(mchy_traces_2.flatten(), bins=100, alpha=0.4, density=True, log=True)
plt.show()

In [ ]:
plt.hist(np.mean(mchy_traces_1, axis=1), bins=100, alpha=0.4, density=True, log=True)
plt.hist(np.mean(mchy_traces_2, axis=1), bins=100, alpha=0.4, density=True, log=True)
plt.show()

In [ ]:
import paulssonlab.deaton.trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib
import dask.dataframe as dd
import pandas as pd

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2021-11-08_lDE20_Final_3/Growth_Division/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_1 = infile["mCherry"][:10] - 200
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/Growth_Division/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_2 = infile["mCherry"][:10] - 200

In [ ]:
stack_1_filt = stack_1[0][stack_1[0] > 1500]
stack_2_filt = stack_2[0][stack_2[0] > 1500]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(stack_1.flatten(), bins=50, range=(0, 6000), alpha=0.5)
plt.hist(stack_2.flatten(), bins=50, range=(0, 6000), alpha=0.5)
plt.show()

In [ ]:
np.mean(stack_1)

In [ ]:
np.mean(stack_2)

In [ ]:
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2021-11-08_lDE20_Final_3/Barcodes/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_1 = infile["Cy7"][:] - 400
with h5py.File(
    "/home/de64/scratch/de64/sync_folder/2022-01-20_lDE20_Final_6/Barcodes/hdf5/hdf5_160.hdf5",
    "r",
) as infile:
    stack_2 = infile["Cy7"][:] - 400

In [ ]:
plt.hist(stack_1.flatten(), bins=50, range=(0, 1000), alpha=0.5)
plt.hist(stack_2.flatten(), bins=50, range=(0, 1000), alpha=0.5)
plt.show()